In [61]:

import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
from scipy.optimize import curve_fit
import numpy as np



def average(list):
    return sum(list)/len(list)

def standarddeviation(list):
    averageofthelist = average(list)
    newlist = [ (elementofthelist - averageofthelist)**2 for elementofthelist in list ]
    sigmasquared  = average(newlist)
    sigma = sigmasquared**0.5
    return sigma

def autocorrelation(list, lag):
    xi = list[:-lag]
    xiplust = list[lag:]
    xitimesxiplust = [ i*iplust for (i, iplust) in zip(xi, xiplust) ]
    averageofxitimexiplust = average(xitimesxiplust)
    averageofxi = average(xi)
    averageofxiplust = average(xiplust)
    autocorrelationvalue = averageofxitimexiplust - (averageofxi*averageofxiplust)
    return autocorrelationvalue

def integratedautocorrelation(list):
    selfcorrelation = standarddeviation(list)
    integratedautocorrelationvalues = [1/2]
    for lag in range(1, len(list)):
        normalisedautocorrelation = autocorrelation(list, lag)/selfcorrelation
        integratedautocorrelationvalues.append(integratedautocorrelationvalues[-1] + normalisedautocorrelation)

    return integratedautocorrelationvalues

def error(list):
    baseerror = standarddeviation(list)/((len(list))**0.5)
    integratedautocorrelationvalues = integratedautocorrelation(list)
    for i in range(1,len(integratedautocorrelationvalues)):
        if integratedautocorrelationvalues[i-1] > integratedautocorrelationvalues[i]:
            newerror = ((2*integratedautocorrelationvalues[i])**0.5)*baseerror
            break
    else:
        newerror = ((2*integratedautocorrelationvalues[-1])**0.5)*baseerror
    
    return newerror

def expFun(x, a, b, c):
    return a*np.exp(-1*b*x) + c*np.ones(len(x))

def plotwitherror(xvals, yvals, err):
    _, ax = plt.subplots()
    parameters, covariance = curve_fit(expFun, xvals, yvals)
    xnew = np.linspace(min(xvals), max(xvals), 300) 
    ynew = expFun(xnew, parameters[0], parameters[1], parameters[2])
    print(parameters)
    ax.plot(xnew, ynew, color='black', linestyle='-.', linewidth=0.75)
    ax.errorbar(xvals, yvals, yerr=err, fmt='.', color='black', linewidth=0.75, capsize=2)
    ax.set_title(r"$\int dt \langle ~\mathrm{Tr}(X(t)X(t)) ~ \mathrm{Tr}(X(t + \Delta t)X(t + \Delta t)) ~ \rangle$ vs $\Delta t$ with Full Action" )
    plt.savefig("fullAction.png", dpi=500)
    plt.show()

In [62]:

def read(file):
    with open(file, 'r') as f:
        data = f.read()
    
    rows = data.split('\n')[32:-1]
    elements = [row.split()[1:] for row in rows]
    corrs = [[] for e in elements[1]]

    for element in elements:
        i=0
        for entry in element:
            corrs[i].append(float(entry))
            i+=1

    averages = []
    for corr in corrs:
        averages.append(average(corr))
    
    return averages
    


In [63]:
nums = [1,2,3,4,5]

fileaverages = []

for num in nums:
    fileaverages.append(read(f'{num}.dat'))

In [64]:
singlecorrlists = [[] for _ in fileaverages[1]]

for file in fileaverages:
    i = 0
    for entry in file:
        singlecorrlists[i].append(entry)
        i = i+1


In [65]:
newdata = [average(singlecorr) for singlecorr in singlecorrlists]
newerrors = [standarddeviation(singlecorr)/(len(singlecorr)**0.5) for singlecorr in singlecorrlists]
t = [1,2,3,4,5,6,7,8]

[ 4.72201968  0.61713724 30.82022787]


In [66]:
plotwitherror(t, newdata, newerrors)